# Final

The final project will consist of a comparison between several CNN architectures for tumor detection. The goal is both to create a high-performing algorithm for differentiating kidneys with tumor from those that are normal, as well as to analyze performance across several different architecture permutations. In total, three different network designs will be tested. As each model is built and trained, ensure to serialize the final model `*.hdf5` file before moving to the next iteration.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model files for all three models
* final compiled `*.csv` file with performance statistics across the different architectures
* final 1-page write-up with methods and results of experiments

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Imports

Use the following lines to import any additional needed libraries:

In [2]:
import numpy as np, pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model, models, layers, losses, metrics, optimizers
from jarvis.train import datasets
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of kidney tumor CT exams derived from the Kidney Tumor Segmentation Challenge (KiTS). More information about the KiTS Challenge can be found here: https://kits21.kits-challenge.org/. The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/ct_kits`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [3]:
# --- Download dataset
datasets.download(name='ct/kits')

{'code': '/data/raw/ct_kits', 'data': '/data/raw/ct_kits'}

Since the algorithms below may require slightly different model inputs, the required generators and inputs will be defined dyanically in the code blocks later in this notebook.

### Data Generator

To accomodate these various permutations, consider the following custom code to implement a nested generator strategy:

In [4]:
def G(gen, dims=2, task='cls', binarize=True):
    """
    Custom generator to modify raw labels for 2D/3D classification or segmentation tasks
    
    :params
    
      (generator) gen      : original unmodified generator
      (int)       dims     : 2D or 3D model
      (str)       task     : 'cls' or 'seg' 
      (bool)      binarize : whether or not to binarize original 3-class labels
    
    """
    assert task in ['cls', 'seg']

    for xs, _ in gen:

        # --- Convert segmentation into classification labels
        if task == 'cls':
            axis = (2, 3, 4) if dims == 2 else (1, 2, 3, 4)
            xs['lbl'] = np.max(xs['lbl'], axis=axis, keepdims=True)
            
        # --- Binarize
        if binarize:
            xs['lbl'] = xs['lbl'] == 2

        yield xs

# Training

A total of three different network architectures will be tested. The goal is to compare the incremental benefit of several design choices. After building and training each model to convergence, do not forget to save each model as a separate `*.hdf5` file.

## 1. Classification

The first task is to create any classification model for binary tumor detection. A 2D model will predict tumor vs. no tumor on a slice-by-slice basis whereas a 3D model will predict tumor vs. no tumor on a volume basis. Regardless of implementation choice, all statistical analysis will be performed on a **volume basis**. For those that choose a 2D model, a reduction strategy must be implemented (see details further below).

In [5]:
def create_blocks(dims=None):
    
    kernel_size = (1, 3, 3) if dims == 2 else (3, 3, 3)
    strides = (1, 2, 2) if dims == 2 else (2, 2, 2)
    
    # --- Define kwargs
    kwargs = {
        'kernel_size': kernel_size,
        'padding': 'same',
        'kernel_initializer': 'he_normal'}

    # --- Define block components
    conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
    tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

    norm = lambda x : layers.BatchNormalization()(x)
    relu = lambda x : layers.ReLU()(x)

    conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
    conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=strides)))
    tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=strides)))

    concat = lambda a, b : layers.Concatenate()([a, b])
                                     
    return conv1, conv2, tran2, concat

In [6]:
def s_e(layer):

  sqz = layers.AveragePooling3D((1, layer.shape[2], layer.shape[3]))(layer)
  cha = int(layer.shape[-1]/4)
  exc = layers.Conv3D(filters = cha, kernel_size = 1, activation = 'relu')(sqz)

  sca = layers.Conv3D(filters = layer.shape[-1], kernel_size = 1, activation = 'sigmoid')(exc)
  return layer * sca

### Create generators

Use the following code cells to choose either a 2D or 3D input. As needed, feel free to modify the batch size and/or implement stratified sampling.

**2D dataset**: To select the 2D data of input size `(1, 96, 96, 1)` use the keyword `2d`:

In [7]:
# --- Prepare generators
configs = {'batch': {'size': 16, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d', configs=configs, custom_layers=True)

**3D dataset**: To select the 3D data of input size `(96, 96, 96, 1)` use the keyword `3d`:

In [26]:
# --- Prepare generators
configs = {'batch': {'size': 2, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='3d', configs=configs, custom_layers=True)

### Define model

In [8]:
conv1, conv2, tran2, concat = create_blocks(dims = 2)

In [9]:
# --- Create backbone model

x = Input(shape=(None, 96, 96, 1), dtype = 'float32')

l1 = conv1(8, x)
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
l6 = conv1(80, conv2(80, l5))

n0, n1, c = l6.shape[-3:]
f0 = layers.Reshape([-1, 1, 1, n0 * n1 * c])(l6)

logits = layers.Conv3D(filters=2, kernel_size=1)(f0)

backbone = Model(inputs = x, outputs= logits)

In [10]:
# --- Create training model

inputs = {
    'dat': Input(shape=(1, 96, 96, 1), name='dat'),
    'lbl': Input(shape=(1, 1, 1, 1), name='lbl')}

logits = backbone(inputs['dat'])

sce = losses.SparseCategoricalCrossentropy(from_logits=True)

loss = sce(y_true=inputs['lbl'], y_pred=logits)

acc = metrics.sparse_categorical_accuracy(y_true=inputs['lbl'], y_pred=logits)

training = Model(inputs=inputs, outputs={'logits': logits, 'loss': loss, 'acc': acc})

training.add_loss(loss)

training.add_metric(acc, name='acc')

optimizer = optimizers.Adam(learning_rate=2e-4)

### Compile and train model

In [11]:
client.load_data_in_memory()

[ 2022-06-06 21:32:37 ] [====================] 100.000% : Iterating | 000402    

In [12]:
# --- Compile model
training.compile(optimizer = optimizer)

# --- Train the model
training.fit(
    x=G(gen_train, dims=2, task='cls'),
    epochs = 30,
    steps_per_epoch = 100,
    validation_data=G(gen_valid, dims=2, task='cls'),
    validation_steps = 100,
    validation_freq = 5
    )

Epoch 1/30
100/100 [==============================] - 8s 35ms/step - loss: 0.6989 - acc: 0.6137
Epoch 2/30
100/100 [==============================] - 4s 35ms/step - loss: 0.6013 - acc: 0.6894
Epoch 3/30
100/100 [==============================] - 3s 29ms/step - loss: 0.5747 - acc: 0.7094
Epoch 4/30
100/100 [==============================] - 2s 23ms/step - loss: 0.4998 - acc: 0.7638
Epoch 5/30
100/100 [==============================] - 4s 43ms/step - loss: 0.5196 - acc: 0.7419 - val_loss: 0.6960 - val_acc: 0.6469
Epoch 6/30
100/100 [==============================] - 2s 23ms/step - loss: 0.4579 - acc: 0.7850
Epoch 7/30
100/100 [==============================] - 2s 22ms/step - loss: 0.4495 - acc: 0.7975
Epoch 8/30
100/100 [==============================] - 2s 22ms/step - loss: 0.4142 - acc: 0.8194
Epoch 9/30
100/100 [==============================] - 2s 22ms/step - loss: 0.3565 - acc: 0.8506
Epoch 10/30
100/100 [==============================] - 5s 49ms/step - loss: 0.3186 - acc: 0.8756 - 

In [14]:
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train = G(test_train, dims=2, task='cls')
test_valid = G(test_valid, dims=2, task='cls')

preds = []
trues = []

for x in test_valid:
    
    pred = backbone.predict(x['dat'])
    preds.append(np.argmax(pred, axis=-1).sum())

    trues.append(x['lbl'].any())

preds = np.array(preds)
trues = np.array(trues)

thresh = np.mean(preds)
preds_ = preds >= thresh


corr = preds_ == trues
tp = np.sum(corr & trues)
tn = np.sum(corr & ~trues)
fn = np.sum(~corr & trues)
fp = np.sum(~corr & ~trues)

acc = (tp + tn) / corr.size
sen = tp / (tp + fn)
spe = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print('Acc: {:0.4f}'.format(acc))
print('Sen: {:0.4f}'.format(sen))
print('Spe: {:0.4f}'.format(spe))
print('PPV: {:0.4f}'.format(ppv))
print('NPV: {:0.4f}'.format(npv))

[ 2022-06-06 21:36:05 ] [====================] 100.000% : Iterating | 000081    Acc: 0.6296
Sen: 0.4681
Spe: 0.8529
PPV: 0.8148
NPV: 0.5370


In [15]:
backbone.save('./wjhan_model1.hdf5')
del backbone

## 2. Segmentation

The second task is to create any segmentation model for binary tumor localization. A 2D model will predict tumor segmentation masks on a slice-by-slice basis whereas a 3D model will predict tumor segmentation masks on a volume basis. Regardless of implementation choice, all statistical analysis will be performed on a **volume basis**. To do so, a reduction strategy must be implemented (see details further below).

### Create generators

Use the following code cells to choose either a 2D or 3D input. As needed, feel free to modify the batch size and/or implement stratified sampling.

**2D dataset**: To select the 2D data of input size `(1, 96, 96, 1)` use the keyword `2d`:

In [17]:
# --- Prepare generators
configs = {'batch': {'size': 16, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d', configs=configs, custom_layers=True)

**3D dataset**: To select the 3D data of input size `(96, 96, 96, 1)` use the keyword `3d`:

In [ ]:
# --- Prepare generators
configs = {'batch': {'size': 2, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='3d', configs=configs, custom_layers=True)

### Define model

In [32]:
conv1, conv2, tran2, concat = create_blocks(dims = 2)

In [33]:
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'
}

In [34]:
# --- Create backbone model

x = Input(shape=(None, 96, 96, 1), dtype='float32')

l1 = conv1(8, x)
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
l6 = tran2(48, l5)
l7  = tran2(32, conv1(48, concat(l4, l6)))
l8  = tran2(16, conv1(32, concat(l3, l7)))
l9  = tran2(8,  conv1(16, concat(l2, l8)))
l10 = conv1(8,  l9)

logits = layers.Conv3D(filters=2, **kwargs)(l10)

backbone = Model(inputs=x, outputs=logits)

In [35]:
inputs = {
    'dat': Input(shape=(None, 96, 96, 1), name='dat'),
    'lbl': Input(shape=(None, 96, 96, 1), name='lbl')}

logits = backbone(inputs['dat'])


In [36]:
# --- Create training model
sce = losses.SparseCategoricalCrossentropy(from_logits=True)
loss = sce(y_true=inputs['lbl'], y_pred=logits)
acc = metrics.sparse_categorical_accuracy(y_true=inputs['lbl'], y_pred=logits)
training = Model(inputs=inputs, outputs={'logits': logits, 'loss': loss, 'acc': acc})
training.add_loss(loss)

training.add_metric(acc, name = 'acc')
optimizer = optimizers.Adam(learning_rate=2e-4)

### Compile and train model

In [37]:
# --- Compile model
training.compile(optimizer = optimizer)

# --- Train the model
training.fit(
    x=G(gen_train, dims=2, task='seg'),
    epochs = 30,
    validation_steps = 100,
    steps_per_epoch = 100,
    validation_freq = 5,
    validation_data=G(gen_valid, dims=2, task='seg')
    )

Epoch 1/30
100/100 [==============================] - 14s 122ms/step - loss: 0.5566 - acc: 0.8074
Epoch 2/30
100/100 [==============================] - 13s 126ms/step - loss: 0.2492 - acc: 0.9748
Epoch 3/30
100/100 [==============================] - 12s 122ms/step - loss: 0.1526 - acc: 0.9770
Epoch 4/30
100/100 [==============================] - 13s 127ms/step - loss: 0.1078 - acc: 0.9792
Epoch 5/30
100/100 [==============================] - 28s 284ms/step - loss: 0.0845 - acc: 0.9808 - val_loss: 0.0938 - val_acc: 0.9768
Epoch 6/30
100/100 [==============================] - 13s 126ms/step - loss: 0.0688 - acc: 0.9825
Epoch 7/30
100/100 [==============================] - 12s 124ms/step - loss: 0.0574 - acc: 0.9844
Epoch 8/30
100/100 [==============================] - 12s 120ms/step - loss: 0.0526 - acc: 0.9844
Epoch 9/30
100/100 [==============================] - 12s 120ms/step - loss: 0.0468 - acc: 0.9861
Epoch 10/30
100/100 [==============================] - 23s 236ms/step - loss: 0.0

In [39]:
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train = G(test_train, dims=2, task='seg')
test_valid = G(test_valid, dims=2, task='seg')

preds = []
trues = []

for x in test_train:
    
    pred = backbone.predict(x['dat'])
    preds.append(np.argmax(pred, axis=-1).sum())

    trues.append(x['lbl'].any())

preds = np.array(preds)
trues = np.array(trues)

thresh = np.mean(preds)
preds_ = preds >= thresh


corr = preds_ == trues
tp = np.sum(corr & trues)
tn = np.sum(corr & ~trues)
fn = np.sum(~corr & trues)
fp = np.sum(~corr & ~trues)

acc = (tp + tn) / corr.size
sen = tp / (tp + fn)
spe = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print('Acc: {:0.4f}'.format(acc))
print('Sen: {:0.4f}'.format(sen))
print('Spe: {:0.4f}'.format(spe))
print('PPV: {:0.4f}'.format(ppv))
print('NPV: {:0.4f}'.format(npv))

[ 2022-06-06 21:56:23 ] [====================] 100.000% : Iterating | 000321    Acc: 0.7383
Sen: 0.4717
Spe: 1.0000
PPV: 1.0000
NPV: 0.6585


In [40]:
backbone.save('./wjhan_model2.hdf5')
del backbone

## 3. Custom architecture

Finally, using any of the customizations described in class, find a top-performing model that may potentially yield some incremental benefit over the two baseline models above.

### Create generators

In [41]:
# --- Choose input (may copy the generator code from above)

configs = {'batch': {'size': 16, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d', configs=configs, custom_layers=True)

### Define model

In [42]:
conv1, conv2, tran2, concat = create_blocks(dims = 2)

In [43]:
# --- Create backbone model

x = Input(shape=(None, 96, 96, 1), dtype = 'float32')

l1 = conv1(8, x)

l2 = conv1(16, conv2(16, l1))
l2 = s_e(l2)

l3 = conv1(32, conv2(32, l2))

l4 = conv1(48, conv2(48, l3))
l4 = s_e(l4)

l5 = conv1(64, conv2(64, l4))

l6 = conv1(80, conv2(80, l5))
l6 = s_e(l6)

n0, n1, c = l6.shape[-3:]
f0 = layers.Reshape([-1, 1, 1, n0 * n1 * c])(l6)

logits = layers.Conv3D(filters=2, kernel_size=1)(f0)

backbone = Model(inputs = x, outputs= logits)

In [44]:
# --- Create training model

inputs = {
    'dat': Input(shape=(1, 96, 96, 1), name='dat'),
    'lbl': Input(shape=(1, 1, 1, 1), name='lbl')}

logits = backbone(inputs['dat'])

sce = losses.SparseCategoricalCrossentropy(from_logits=True)

loss = sce(y_true=inputs['lbl'], y_pred=logits)

acc = metrics.sparse_categorical_accuracy(y_true=inputs['lbl'], y_pred=logits)

training = Model(inputs=inputs, outputs={'logits': logits, 'loss': loss, 'acc': acc})

training.add_loss(loss)

training.add_metric(acc, name='acc')

optimizer = optimizers.Adam(learning_rate=2e-4)

### Compile and train model

In [48]:
# --- Compile model
training.compile(optimizer = optimizer)

# --- Train the model
training.fit(
    x=G(gen_train, dims=2, task='cls'),
    epochs = 30,
    steps_per_epoch = 100,
    validation_data=G(gen_valid, dims=2, task='cls'),
    validation_steps = 100,
    validation_freq = 5
    )

Epoch 1/30
100/100 [==============================] - 13s 122ms/step - loss: 0.1052 - acc: 0.9631
Epoch 2/30
100/100 [==============================] - 12s 120ms/step - loss: 0.0847 - acc: 0.9650
Epoch 3/30
100/100 [==============================] - 12s 122ms/step - loss: 0.0798 - acc: 0.9700
Epoch 4/30
100/100 [==============================] - 12s 123ms/step - loss: 0.0735 - acc: 0.9737
Epoch 5/30
100/100 [==============================] - 25s 250ms/step - loss: 0.0762 - acc: 0.9700 - val_loss: 1.3903 - val_acc: 0.6925
Epoch 6/30
100/100 [==============================] - 12s 118ms/step - loss: 0.0587 - acc: 0.9800
Epoch 7/30
100/100 [==============================] - 12s 118ms/step - loss: 0.0551 - acc: 0.9806
Epoch 8/30
100/100 [==============================] - 12s 116ms/step - loss: 0.0557 - acc: 0.9806
Epoch 9/30
100/100 [==============================] - 13s 125ms/step - loss: 0.0635 - acc: 0.9756
Epoch 10/30
100/100 [==============================] - 24s 241ms/step - loss: 0.0

In [50]:
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train = G(test_train, dims=2, task='seg')
test_valid = G(test_valid, dims=2, task='seg')

preds = []
trues = []

for x in test_train:
    
    pred = backbone.predict(x['dat'])
    preds.append(np.argmax(pred, axis=-1).sum())

    trues.append(x['lbl'].any())

preds = np.array(preds)
trues = np.array(trues)

thresh = np.mean(preds)
preds_ = preds >= thresh


corr = preds_ == trues
tp = np.sum(corr & trues)
tn = np.sum(corr & ~trues)
fn = np.sum(~corr & trues)
fp = np.sum(~corr & ~trues)

acc = (tp + tn) / corr.size
sen = tp / (tp + fn)
spe = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print('Acc: {:0.4f}'.format(acc))
print('Sen: {:0.4f}'.format(sen))
print('Spe: {:0.4f}'.format(spe))
print('PPV: {:0.4f}'.format(ppv))
print('NPV: {:0.4f}'.format(npv))

[ 2022-06-06 22:17:01 ] [====================] 100.000% : Iterating | 000321    Acc: 0.9502
Sen: 0.8994
Spe: 1.0000
PPV: 1.0000
NPV: 0.9101


In [51]:
backbone.save('./wjhan_model3.hdf5')
del backbone

# Evaluation

For each of the three models, the following metrics should be calculated for **both the training and validation** cohorts:

* accuracy
* sensitivity
* specificity
* positive predictive value (PPV)
* negative predictive value (NPV)

As in prior assignments, accuracy is determined on a patient by patient (volume by volume) basis, so please implement a prediction reduction strategy as needed for your models.

### Performance

The following minimum **validation cohort** performance metrics must be met for full credit:

1. **Classification**: accuracy > 0.55
2. **Segmentation**: accuracy > 0.55
3. **Custom architecture**: accuracy > 0.60

**Bonus**: the top three overall models based on **validation cohort** accuracy will recieve a +5 point (+15%) extra credit towards the final assignment.

### Results

When ready, create a `*.csv` file with your compiled **training and validation** cohort statistics for the three different models. Consider the following table format (although any format that contains the required information is sufficient):

```
          TRAINING                              VALIDATION
          accuracy | sens | spec | PPV |  NPV | accuracy | sens | spec | PPV |  NPV
model 1
model 2
model 3
```

As above, statistics for both training and validation should be provided.

In [52]:
# --- Create *.csv
model_eval = {'train accuracy': [0.9315, 0.7383, 0.9502],
 'train sens'    : [0.8616, 0.4717, 0.8994],
 'train spec'    : [1.0000, 1.0000, 1.0000],
 'train PPV'     : [1.0000, 1.0000, 1.0000],
 'train NPV'     : [0.8804, 0.6585, 0.9101],
 'valid accuracy': [0.6296, 0.5802,0.6420],
 'valid sens'    : [0.4681, 0.3191, 0.4681],
 'valid spec'    : [0.8529, 0.9412, 0.8824],
 'valid PPV'     : [0.8148, 0.8824, 0.8462],
 'valid NPV'     : [0.5370, 0.5000, 0.5455]
}

df = pd.DataFrame(model_eval, index = ['model1', 'model2', 'model3'])               
# --- Serialize *.csv
df.to_csv('./wjhan_results.csv')

# Summary

In addition to algorithm training as above, a 1-2 page write-up is required for this project. The goal is to *briefly* summarize algorithm design and key results. The write-up should be divided into three sections: methods; results; discussion. More detailed information and tips can be found here: https://github.com/peterchang77/dl_tutor/blob/master/cs190/spring_2021/notebooks/midterm/checklist.md.

### Methods

In this section, include details such as:

* **Data**: How much data was used. How many cases were utilized for training and validation?
* **Network design**: What are the different network architectures? How many layers and parameters? Were 2D or 3D operations used? Recall that the `model.summary(...)` can be used to provide key summary statistics for this purpose. If desired, feel free to include a model figure or diagram.
* **Implementation**: How was training implemented. What are the key hyperparameters (e.g. learning rate, batch size, optimizer, etc)? How many training iterations were required for convergence? Did these hyperparameters change during the course of training?
* **Statistics**: What statistics do you plan to use to evaluate model accuracy? 

### Results

In this section, briefly summarize experimental results (a few sentences), and include the result table(s) as derived above.

### Discussion

Were the results expected or unexpected? What accounts for the differences in performance between the algorithms? How did you choose the network architecture implemented in your final model? Feel free to elaborate on any additional observations noted during the course of this expierment.

# Submission


### Canvas

Once you have completed the midterm assignment, download the necessary files from Google Colab and your Google Drive. As in prior assigments, be sure to prepare:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv` (compiled for all three parts)
* final (trained) model: `[UCInetID]_model.hdf5` (three separate files for all three parts)

In addition, submit the summary write-up as in any common document format (`.docx`, `.tex`, `.pdf`, etc):

* final summary write-up: `[UCInetID]_summary.[docx|tex|pdf]`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadsheet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.